In [53]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm 
from mpl_toolkits.mplot3d import axes3d
from numpy.random import multivariate_normal
import copy
from functools import partial

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

try:
    import seaborn as sns
    sns.set_style("whitegrid")
    sns.set_context('talk')
    #sns.set(font_scale=1.4)
except ImportError:
    plt.style.use('seaborn-whitegrid')

In [54]:
class CMAES:
    
    def __init__(self, initial_mean, sigma, popsize, **kwargs):

        # Things that evolve : centroid, sigma, paths etc.
        self.centroid = np.asarray(initial_mean).copy()
        self.sigma = sigma
        self.pc = np.zeros_like(initial_mean)
        self.ps = np.zeros_like(initial_mean)        
        self.C = np.eye(initial_mean.shape[0])
        self.B = np.eye(self.C.shape[0])
        self.diagD = np.ones(initial_mean.shape[0])
        
        # Optimal popsize 
        self.popsize = popsize
        self.mu = popsize // 2
        
        # Update weights later on
        # Constant weight policy
        # self.weights = np.ones((self.mu, )) / self.mu

        # Decreasing weight policy
        self.weights = np.arange(self.mu, 0.0, -1.0)
        self.weights /= np.sum(self.weights)
        
        # Negative, Positive weight policy
        # unscaled_weights = np.arange(1.0 ,  1.0 + popsize)
        # unscaled_weights = np.log(0.5 * (popsize + 1.0) / unscaled_weights)

        # Utility variables
        self.dim = initial_mean.shape[0]

        # Expectation of a normal distribution
        self.chiN = np.sqrt(self.dim) * (1.0 - 0.25 / self.dim + 1.0/(21.0 * self.dim**2))
        self.mueff = 1.0 / np.linalg.norm(self.weights, 2)**2
        self.generations = 0
        
        # Options
 
        # Sigma adaptation
        # cs is short for c_sigma
        self.cs = kwargs.get("cs", (2.0 + self.mueff) / (self.dim + self.mueff + 5.0))
        # ds is short for d_sigma
        self.ds = 1.0 + 2.0 * max(0.0, np.sqrt((self.mueff - 1.0)/ (self.dim + 1.0)) - 1.0) + self.cs
        
        # Covariance adaptation
        self.cc = kwargs.get("cc", (4.0 + self.mueff/self.dim) / (self.dim + 4.0 + 2.0 * self.mueff/self.dim))
        self.ccov = 0.0
        # If implementing the latest version of CMA according to the tutorial, 
        # these parameters can be useful
        self.ccov1 = 2.0 / ((self.dim + 1.3)**2 + self.mueff)
        self.ccovmu = min(1.0 - self.ccov1, 2.0 * (self.mueff - 2.0 + 1.0/self.mueff)/((self.dim + 2.0)**2 + self.mueff))
        
          # asserts to guide you
#         assert self.dim == 2, "We are dealing with a two-dimensional problem only"
#         assert self.centroid.shape == (2,), "Centroid shape is incorrect, did you tranpose it by mistake?"
#         assert self.sigma > 0.0, "Sigma is not a non-zero positive number!"
#         assert self.pc.shape == (2, ), "pc shape is incorrect, did you tranpose it by mistake?"
#         assert self.ps.shape == (2, ), "ps shape is incorrect, did you tranpose it by mistake?"
#         assert self.C.shape == (2, 2), "C's shape is incorrect, remember C is a matrix!"     
#         assert type(self.popsize) == int, "Population size not an integer"
#         assert self.popsize > 0 , "Population size is negative!"
#         assert self.popsize > 2 , "Too little population size, make it >2"
        
        self.stats_centroids = []
        self.stats_new_centroids = []
        self.stats_covs = []
        self.stats_new_covs = []
        self.stats_offspring = []
        self.stats_offspring_weights = []
        self.stats_ps = []
    
    def update(self, problem, population):
        """Update the current covariance matrix strategy from the
        *population*.
        
        :param population: A list of individuals from which to update the
                           parameters.
        """
        # -- store current state of the algorithm
        self.stats_centroids.append(copy.deepcopy(self.centroid))
        self.stats_covs.append(copy.deepcopy(self.C))
        
        print(problem(population))
        population = sorted(population, key = lambda ind: problem(ind), reverse = True)
        print(problem(population))
        # population.sort(key=lambda ind: problem(ind[0], ind[1]))
        # population.sort(key=problem)
        
        # -- store sorted offspring
        self.stats_offspring.append(copy.deepcopy(population))
        
        old_centroid = self.centroid
        # Note : the following does m <- <x>_w
        # Note : this is equivalent to doing m <- m + sigma * <z>_w
        # as x = m + sigma * z provided the weights sum to 1.0 which it
        # does
        self.centroid = np.dot(self.weights, population[0:self.mu])
        
        # -- store new centroid
        self.stats_new_centroids.append(copy.deepcopy(self.centroid))
        
        c_diff = self.centroid - old_centroid
        
        # Cumulation : update evolution path
        # Equivalent to in-class definition
        self.ps = (1 - self.cs) * self.ps \
             + np.sqrt(self.cs * (2 - self.cs) * self.mueff) / self.sigma \
             * np.dot(self.B, (1. / self.diagD) * np.dot(self.B.T, c_diff))
        
        # -- store new evol path
        self.stats_ps.append(copy.deepcopy(self.ps))
        
        hsig = float((np.linalg.norm(self.ps) / 
                np.sqrt(1. - (1. - self.cs)**(2. * (self.generations + 1.))) / self.chiN
                < (1.4 + 2. / (self.dim + 1.))))
        
        self.pc = (1 - self.cc) * self.pc + hsig \
                  * np.sqrt(self.cc * (2 - self.cc) * self.mueff) / self.sigma \
                  * c_diff
        
        # Update covariance matrix
        artmp = population[0:self.mu] - old_centroid
        self.C = (1 - self.ccov1 - self.ccovmu + (1 - hsig) \
                   * self.ccov1 * self.cc * (2 - self.cc)) * self.C \
                + self.ccov1 * np.outer(self.pc, self.pc) \
                + self.ccovmu * np.dot((self.weights * artmp.T), artmp) \
                / self.sigma**2
        
        # -- store new covs
        self.stats_new_covs.append(copy.deepcopy(self.C))
        
        self.sigma *= np.exp((np.linalg.norm(self.ps) / self.chiN - 1.) \
                                * self.cs / self.ds)
        
        self.diagD, self.B = np.linalg.eigh(self.C)
        indx = np.argsort(self.diagD)
        
        self.cond = self.diagD[indx[-1]]/self.diagD[indx[0]]
        
        self.diagD = self.diagD[indx]**0.5
        self.B = self.B[:, indx]
        self.BD = self.B * self.diagD
        
    def calc_weight(self, pop, item_weights):
        weights = pop @ item_weights
        return weights
    
    # eliminate members that don't meet weight constraint
    def weight_constraint(self,pop, capacity, item_weights):
        idx = (self.calc_weight(pop, item_weights) <= capacity)
        return pop[idx]
        
    def run(self, problem, data_set):
        
        n_items = int(data_set['n_items'])
        bag_capacity = int(data_set['capacity'])
        item_values = data_set['item_values']
        item_weights = data_set['item_weights']
        
        # At the start, clear all stored cache and start a new campaign
        self.reset()
        
        while self.generations < 40:
            # Sample the population here!
            population = np.absolute(np.round(np.random.multivariate_normal(self.centroid, self.sigma**2 * self.C, self.popsize)))
            print(population[:,0].size)
            population = self.weight_constraint(population, bag_capacity, item_weights)
            print(population[:,0].size)
            while population[:,0].size < self.popsize:
                n = self.popsize-population[:,0].size
                new_members = np.absolute(np.round(multivariate_normal(self.centroid, self.sigma**2 * self.C, n)))
                population = np.vstack((population,new_members))
                population = self.weight_constraint(population, bag_capacity, item_weights)
            print(population[:,0].size)
            print()

            # Pass the population to update, which computes all new parameters 
            self.update(problem, population)
            # print(np.array(population).shape)
            self.generations += 1
        else:
            return population
        
    def reset(self):
        # Clears everything to rerun the problem
        self.stats_centroids = []
        self.stats_new_centroids = []
        self.stats_covs = []
        self.stats_new_covs = []
        self.stats_offspring = []
        self.stats_offspring_weights = []
        self.stats_ps = []

In [55]:
def knapsack_value(item_vals,population):
    return population @ item_vals

In [56]:
# dataset A: algorithm to find optimal solution
my_data = np.load('A.npz')
bag_capacity = int(my_data['capacity'])
n_items = int(my_data['n_items'])
item_values = my_data['item_values']
item_weights = my_data['item_weights']
initial_centroid_A = np.zeros(n_items,) 
knapsack_A = partial(knapsack_value, np.load('A.npz')['item_values'])

cma_es = CMAES(initial_centroid_A, 0.3, 50)
final_pop = cma_es.run(knapsack_A, np.load('A.npz'))
print(knapsack_A(final_pop))

50
23
50

[22. 16. 13. 17.  9. 27. 10. 25. 10. 17. 37. 30. 12. 26. 30. 10. 18.  0.
 21. 14. 21. 20.  4. 31. 10. 10. 10. 15.  7. 12.  6. 12.  8. 30. 18.  9.
 27. 26. 41. 13.  9. 11. 17. 13. 17. 26. 18. 32. 23. 12.]
[41. 37. 32. 31. 30. 30. 30. 27. 27. 26. 26. 26. 25. 23. 22. 21. 21. 20.
 18. 18. 18. 17. 17. 17. 17. 16. 15. 14. 13. 13. 13. 12. 12. 12. 12. 11.
 10. 10. 10. 10. 10. 10.  9.  9.  9.  8.  7.  6.  4.  0.]
50
6
50

[34. 23. 22. 33. 25. 14. 25. 18. 25. 27. 26. 20. 47. 21. 27. 19. 49. 39.
 20. 31. 30. 12. 27. 23. 26. 30. 37. 17. 25. 45. 41. 58. 16. 31. 26. 18.
 26. 34. 56. 18. 13. 20. 24. 13. 32. 20. 29. 52.  7.  6.]
[58. 56. 52. 49. 47. 45. 41. 39. 37. 34. 34. 33. 32. 31. 31. 30. 30. 29.
 27. 27. 27. 26. 26. 26. 26. 25. 25. 25. 25. 24. 23. 23. 22. 21. 20. 20.
 20. 20. 19. 18. 18. 18. 17. 16. 14. 13. 13. 12.  7.  6.]
50
1
50

[18. 24. 36. 27. 54. 29. 37. 26. 40. 31. 29. 43. 35. 35. 32. 35. 31. 11.
 36. 19. 18. 36. 34. 50. 24. 49. 23. 20. 36. 42. 42. 21. 20.  7. 31. 34.
 32. 48. 5

50

[ 86.  82.  79.  84.  98.  83.  93.  94.  86.  67.  85.  90.  90.  84.
  76.  86.  60. 102.  81.  94.  86. 100.  84.  71.  99.  84.  95.  96.
  89.  74.  89.  76.  82.  92.  93.  94.  76.  86.  84. 101.  86.  99.
  99.  84.  80.  95.  88. 107.  81.  99.]
[107. 102. 101. 100.  99.  99.  99.  99.  98.  96.  95.  95.  94.  94.
  94.  93.  93.  92.  90.  90.  89.  89.  88.  86.  86.  86.  86.  86.
  86.  85.  84.  84.  84.  84.  84.  84.  83.  82.  82.  81.  81.  80.
  79.  76.  76.  76.  74.  71.  67.  60.]
50
0
50

[ 86.  90.  87.  87.  81. 104.  80.  94.  95.  52.  79.  82. 100.  78.
  89.  84.  89.  92.  99.  77.  90.  82. 102.  79.  89.  84.  90.  77.
 104.  77. 100.  66.  83.  93. 109.  94.  93.  90.  86.  87.  99. 110.
  80.  82.  80.  92. 106.  81.  78.  89.]
[110. 109. 106. 104. 104. 102. 100. 100.  99.  99.  95.  94.  94.  93.
  93.  92.  92.  90.  90.  90.  90.  89.  89.  89.  89.  87.  87.  87.
  86.  86.  84.  84.  83.  82.  82.  82.  81.  81.  80.  80.  80.  79.
  79.  78

50

[137. 137. 127. 138. 117. 128. 136. 132. 130. 126. 117. 129. 134. 120.
 125. 139. 139. 125. 143. 127. 120. 118. 132. 134. 142. 125. 120. 130.
 139. 120. 148. 124. 131. 135. 126. 143. 117. 126. 136. 128. 137. 129.
 135. 134. 125. 130. 111. 132. 138. 124.]
[148. 143. 143. 142. 139. 139. 139. 138. 138. 137. 137. 137. 136. 136.
 135. 135. 134. 134. 134. 132. 132. 132. 131. 130. 130. 130. 129. 129.
 128. 128. 127. 127. 126. 126. 126. 125. 125. 125. 125. 124. 124. 120.
 120. 120. 120. 118. 117. 117. 117. 111.]
50
3
50

[144. 124. 126. 139. 140. 136. 136. 138. 131. 137. 136. 137. 128. 135.
 152. 132. 139. 124. 148. 139. 135. 129. 131. 128. 139. 142. 134. 135.
 140. 142. 121. 138. 125. 142. 143. 143. 123. 118. 129. 120. 146. 110.
 142. 133. 124. 137. 131. 129. 136. 144.]
[152. 148. 146. 144. 144. 143. 143. 142. 142. 142. 142. 140. 140. 139.
 139. 139. 139. 138. 138. 137. 137. 137. 136. 136. 136. 136. 135. 135.
 135. 134. 133. 132. 131. 131. 131. 129. 129. 129. 128. 128. 126. 125.
 124. 124